<a href="https://colab.research.google.com/github/JimMcDonough/Capstone/blob/main/Bert_Hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Nov 11 23:10:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    42W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 73.4 MB/s 
     |████████████████████████████████| 163 kB 90.1 MB/s 


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.8 MB/s 
     |████████████████████████████████| 115 kB 81.4 MB/s 
     |████████████████████████████████| 212 kB 79.7 MB/s 
     |████████████████████████████████| 95 kB 3.7 MB/s 
     |████████████████████████████████| 127 kB 86.7 MB/s 
     |████████████████████████████████| 115 kB 82.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from datasets import load_dataset, Value, Sequence, Features

data_files={"train": r"/content/drive/My Drive/sample_data/train.json",
           "validation": r"/content/drive/My Drive/sample_data/val.json",
           "test": r"/content/drive/My Drive/sample_data/test.json"}

In [ ]:
features = Features({'answers': [{'text': Value(dtype='string'), 'answer_start': Value(dtype='int32')}],
'context': Value(dtype='string'),
'id': Value(dtype='string'),
'question': Value(dtype='string'),
'title': Value(dtype='string')})

In [ ]:
dataset = load_dataset("json", data_files=data_files, features=features, cache_dir= r"/content/drive/My Drive/sample_data/")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",     #only truncates the context nothing else
        return_offsets_mapping=True,  # map the start and end positions of the answer to the original context 
        padding="max_length",
    )
    
    offset_mapping = inputs.pop("offset_mapping")
    #print(offset_mapping)
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    #print(answers)
    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        #start_char = answer["answer_start"][0]
        #end_char = answer["answer_start"][0] + len(answer["text"][0])  #start char + length giver chars with answer
        start_char = answer[0]["answer_start"]
        end_char = answer[0]["answer_start"] + len(answer[0]["text"])  
        sequence_ids = inputs.sequence_ids(i)  #no clue what this does
        #print(answer, end_char, sequence_ids)
        # Find the start and end of the context        ; not sure how this section works
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
    
        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
            
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
            
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
from transformers import DefaultDataCollator
from transformers import TFAutoModelForQuestionAnswering
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

data_collator = DefaultDataCollator(return_tensors="tf")

In [ ]:
batch_sizes = [8,16]
epoch_sizes = [3]
learning_rates = [0.00032, 0.000192, 0.000128, 0.000064]

In [ ]:
results_list = []
for num_epochs in epoch_sizes:
    for batch_size in batch_sizes:
        for learning_rate in learning_rates:
            result = {'epochs':num_epochs,
                     'batch_size': batch_size,
                     'learning_rate':learning_rate}
            
            model = TFAutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")
            
            tf_train_set = model.prepare_tf_dataset(
                tokenized_squad["train"],
                shuffle=True,
                batch_size=batch_size,
                collate_fn=data_collator,
            )
            
            tf_validation_set = model.prepare_tf_dataset(
                tokenized_squad["validation"],
                shuffle=False,
                batch_size=batch_size,
                collate_fn=data_collator,
            )
            
            num_train_steps = len(tf_train_set) * num_epochs
            lr_scheduler = PolynomialDecay(
                initial_learning_rate=learning_rate, end_learning_rate=0.0, decay_steps=num_train_steps
            )
            
            opt = Adam(learning_rate=lr_scheduler)
            
            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
            model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
            
            history = model.fit(tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)
            result['val_end_accuracy'] = history.history['val_end_logits_accuracy']
            result['val_start_accuracy'] = history.history['val_start_logits_accuracy']
            results_list.append(result) 
            del history
            del model
            del tf_train_set
            del tf_validation_set

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1575/1575 [==============================] - 181s 106ms/step - loss: 9.9508 - end_logits_loss: 4.9327 - start_logits_loss: 5.0181 - end_logits_accuracy: 0.0561 - start_logits_accuracy: 0.0550 - val_loss: 11.9013 - val_end_logits_loss: 5.9507 - val_start_logits_loss: 5.9507 - val_end_logits_accuracy: 0.0032 - val_start_logits_accuracy: 0.0030
Epoch 2/3
1575/1575 [==============================] - 164s 104ms/step - loss: 11.9097 - end_logits_loss: 5.9540 - start_logits_loss: 5.9557 - end_logits_accuracy: 0.0022 - start_logits_accuracy: 0.0032 - val_loss: 11.9013 - val_end_logits_loss: 5.9507 - val_start_logits_loss: 5.9507 - val_end_logits_accuracy: 0.0040 - val_start_logits_accuracy: 0.0034
Epoch 3/3
1575/1575 [==============================] - 164s 104ms/step - loss: 11.9064 - end_logits_loss: 5.9530 - start_logits_loss: 5.9534 - end_logits_accuracy: 0.0025 - start_logits_accuracy: 0.0030 - val_loss: 11.9013 - val_end_logits_loss: 5.9507 - val_start_logits_loss: 5.9507 - val_

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1575/1575 [==============================] - 179s 107ms/step - loss: 4.6830 - end_logits_loss: 2.2708 - start_logits_loss: 2.4122 - end_logits_accuracy: 0.4163 - start_logits_accuracy: 0.3933 - val_loss: 3.7217 - val_end_logits_loss: 1.7908 - val_start_logits_loss: 1.9309 - val_end_logits_accuracy: 0.5365 - val_start_logits_accuracy: 0.4998
Epoch 2/3
1575/1575 [==============================] - 164s 104ms/step - loss: 2.5272 - end_logits_loss: 1.2265 - start_logits_loss: 1.3008 - end_logits_accuracy: 0.6516 - start_logits_accuracy: 0.6326 - val_loss: 3.4268 - val_end_logits_loss: 1.6049 - val_start_logits_loss: 1.8219 - val_end_logits_accuracy: 0.5887 - val_start_logits_accuracy: 0.5516
Epoch 3/3
1575/1575 [==============================] - 164s 104ms/step - loss: 1.0735 - end_logits_loss: 0.5106 - start_logits_loss: 0.5629 - end_logits_accuracy: 0.8406 - start_logits_accuracy: 0.8187 - val_loss: 3.5787 - val_end_logits_loss: 1.6872 - val_start_logits_loss: 1.8915 - val_end_l

...

Epoch 1/3
787/787 [==============================] - 158s 188ms/step - loss: 4.0407 - end_logits_loss: 1.9796 - start_logits_loss: 2.0611 - end_logits_accuracy: 0.4859 - start_logits_accuracy: 0.4626 - val_loss: 3.2703 - val_end_logits_loss: 1.5620 - val_start_logits_loss: 1.7083 - val_end_logits_accuracy: 0.5865 - val_start_logits_accuracy: 0.5498
Epoch 2/3
787/787 [==============================] - 145s 184ms/step - loss: 1.9106 - end_logits_loss: 0.9359 - start_logits_loss: 0.9747 - end_logits_accuracy: 0.7270 - start_logits_accuracy: 0.7032 - val_loss: 3.0490 - val_end_logits_loss: 1.4777 - val_start_logits_loss: 1.5712 - val_end_logits_accuracy: 0.6391 - val_start_logits_accuracy: 0.5899
Epoch 3/3
787/787 [==============================] - 145s 184ms/step - loss: 0.7650 - end_logits_loss: 0.3760 - start_logits_loss: 0.3890 - end_logits_accuracy: 0.8810 - start_logits_accuracy: 0.8673 - val_loss: 3.6005 - val_end_logits_loss: 1.6820 - val_start_logits_loss: 1.9185 - val_end_logits_

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
787/787 [==============================] - 158s 188ms/step - loss: 3.5099 - end_logits_loss: 1.7291 - start_logits_loss: 1.7808 - end_logits_accuracy: 0.5440 - start_logits_accuracy: 0.5164 - val_loss: 2.8475 - val_end_logits_loss: 1.3470 - val_start_logits_loss: 1.5005 - val_end_logits_accuracy: 0.6448 - val_start_logits_accuracy: 0.5885
Epoch 2/3
787/787 [==============================] - 144s 184ms/step - loss: 1.5814 - end_logits_loss: 0.7756 - start_logits_loss: 0.8058 - end_logits_accuracy: 0.7715 - start_logits_accuracy: 0.7474 - val_loss: 2.9058 - val_end_logits_loss: 1.3586 - val_start_logits_loss: 1.5473 - val_end_logits_accuracy: 0.6695 - val_start_logits_accuracy: 0.6201
Epoch 3/3
787/787 [==============================] - 144s 183ms/step - loss: 0.6522 - end_logits_loss: 0.3249 - start_logits_loss: 0.3273 - end_logits_accuracy: 0.8976 - start_logits_accuracy: 0.8871 - val_loss: 3.5020 - val_end_logits_loss: 1.6350 - val_start_logits_loss: 1.8670 - val_end_logits_

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
787/787 [==============================] - 157s 187ms/step - loss: 3.5189 - end_logits_loss: 1.7259 - start_logits_loss: 1.7930 - end_logits_accuracy: 0.5545 - start_logits_accuracy: 0.5235 - val_loss: 2.6499 - val_end_logits_loss: 1.2426 - val_start_logits_loss: 1.4073 - val_end_logits_accuracy: 0.6610 - val_start_logits_accuracy: 0.6037
Epoch 2/3
787/787 [==============================] - 144s 183ms/step - loss: 1.5172 - end_logits_loss: 0.7390 - start_logits_loss: 0.7782 - end_logits_accuracy: 0.7821 - start_logits_accuracy: 0.7543 - val_loss: 2.8188 - val_end_logits_loss: 1.3257 - val_start_logits_loss: 1.4930 - val_end_logits_accuracy: 0.6652 - val_start_logits_accuracy: 0.6110
Epoch 3/3
787/787 [==============================] - 144s 183ms/step - loss: 0.7312 - end_logits_loss: 0.3581 - start_logits_loss: 0.3732 - end_logits_accuracy: 0.8844 - start_logits_accuracy: 0.8732 - val_loss: 3.2219 - val_end_logits_loss: 1.5125 - val_start_logits_loss: 1.7094 - val_end_logits_